In [43]:
import json
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D
import itertools
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

import progressbar
from metrics.accuracy import conlleval

In [44]:
def read_data(fname):
    text_l = []
    label_l = []
    with open(fname) as f:
        for line1,line2 in itertools.izip_longest(*[f]*2):
            try:
                text = ast.literal_eval(line1)
                label = ast.literal_eval(line2)
                text_l.append(text)
                label_l.append(label)
            except:
                pass
    f.close()
    return text_l, label_l

In [45]:
X_text, y_text = read_data('../data/11532192.txt')

text = list(set(itertools.chain(*X_text)))

le = LabelEncoder()
le.fit(text)

w2idx = dict(zip(le.classes_, le.transform(le.classes_)))

idx2w  = {w2idx[k]:k for k in w2idx}


X = []
for each in X_text:
    X.append(le.transform(each))
    
label = list(set(itertools.chain(*y_text)))

le = LabelEncoder()
le.fit(label)

labels2idx = dict(zip(le.classes_, le.transform(le.classes_)))

idx2la = {labels2idx[k]:k for k in labels2idx}

y = []
for each in y_text:
    y.append(le.transform(each))

# w2idx, labels2idx = dicts['words2idx'], dicts['labels2idx']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

words_val = [ list(map(lambda x: idx2w[x], w)) for w in X_test]
groundtruth_val = [ list(map(lambda x: idx2la[x], y)) for y in y_test] # y values test
words_train = [ list(map(lambda x: idx2w[x], w)) for w in X_train]
groundtruth_train = [ list(map(lambda x: idx2la[x], y)) for y in y_train] # y values train

n_classes = len(idx2la)
n_vocab = len(idx2w)

In [46]:
print("Example sentence : {}".format(words_train[0]))
print("Encoded form: {}".format(X_train[0]))
print()
print("It's label : {}".format(groundtruth_val[0]))
print("Encoded form: {}".format(y_train[0]))

Example sentence : ['In', 'strain', '129P3J', 'IOP', 'did', 'not', 'differ', 'significantly', 'with', 'age', 'between', '3', 'and', '14', 'months', 'but', 'was', 'lower', 'in', '18', 'month', 'old', 'mice', 'P', 'lt', '0001', 'compared', 'to', 'all', 'younger', 'ages', 'Figure', '4', 'Despite', 'a', '1', 'mmHg', 'dip', 'in', 'IOP', 'at', '8', 'months', 'there', 'were', 'no', 'significant', 'IOP', 'differences', 'between', 'C3HHeJ', 'mice', 'at', 'each', 'age', 'tested', 'P', '', '02', 'for', 'age', 'Although', 'the', 'effect', 'of', 'age', 'has', 'not', 'been', 'thoroughly', 'assessed', 'in', 'other', 'strains', 'no', 'obvious', 'agerelated', 'differences', 'have', 'been', 'identified', 'in', 'other', 'strains', 'analyzed', 'at', 'multiple', 'ages', 'except', 'for', 'the', 'glaucomatous', 'DBA2J', 'and', 'AKXD28Ty', 'strains']
Encoded form: [ 227 1095   31  222  582  887  584 1073 1204  361  443   75  396   43  864
  454 1187  815  749   54  863  907  849  280  817    3  501 1146  371 

In [47]:
print('Build model...')
model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Convolution1D(64,5,border_mode='same', activation='relu'))
model.add(Dropout(0.25))
model.add(GRU(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')

Build model...


/Users/somya/Virtualenvs/ml/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`


In [49]:
### Training
n_epochs = 100

train_f_scores = []
val_f_scores = []
best_val_f1 = 0

for i in range(n_epochs):
    print("Epoch {}".format(i))
    
    print("Training =>")
    train_pred_label = []
    avgLoss = 0


    bar = progressbar.ProgressBar(maxval=len(X_train))
    for n_batch, sent in bar(enumerate(X_train)):
        print n_batch, sent
        label = y_train[n_batch]
        label = np.eye(n_classes)[label][np.newaxis,:]
        sent = sent[np.newaxis,:]

        if sent.shape[1] > 1: #some bug in keras
            loss = model.train_on_batch(sent, label)
            avgLoss += loss

        pred = model.predict_on_batch(sent)
        pred = np.argmax(pred,-1)[0]
        train_pred_label.append(pred)

    avgLoss = avgLoss/n_batch
    
    predword_train = [ list(map(lambda x: idx2la[x], y)) for y in train_pred_label]
    
    
#     con_dict = conlleval(predword_train, groundtruth_train, words_train, 'r.txt')
#     print con_dict
#     train_f_scores.append(con_dict['f1'])
    

    print classification_report(list(itertools.chain.from_iterable(groundtruth_train)), 
                                list(itertools.chain.from_iterable(predword_train)))
    
#     print('Loss = {}, Precision = {}, Recall = {}, F1 = {}'.format(avgLoss, con_dict['r'], con_dict['p'], con_dict['f1']))
    
    
    print("Validating =>")
    
    val_pred_label = []
    avgLoss = 0
    
    bar = progressbar.ProgressBar(maxval=len(X_test))
    for n_batch, sent in bar(enumerate(X_test)):
        label = y_test[n_batch]
        label = np.eye(n_classes)[label][np.newaxis,:]
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #some bug in keras
            loss = model.test_on_batch(sent, label)
            avgLoss += loss

        pred = model.predict_on_batch(sent)
        pred = np.argmax(pred,-1)[0]
        val_pred_label.append(pred)

    avgLoss = avgLoss/n_batch
    
    predword_val = [ list(map(lambda x: idx2la[x], y)) for y in val_pred_label]
#     con_dict = conlleval(predword_val, groundtruth_val, words_val, 'r.txt')
#     print con_dict
#     val_f_scores.append(con_dict['f1'])
    
#     print('Loss = {}, Precision = {}, Recall = {}, F1 = {}'.format(avgLoss, con_dict['r'], con_dict['p'], con_dict['f1']))

    print classification_report(list(itertools.chain.from_iterable(groundtruth_val)), 
                                list(itertools.chain.from_iterable(predword_val)))

    
#     if con_dict['f1'] > best_val_f1:
#     	best_val_f1 = con_dict['f1']
#     	open('model_architecture.json','w').write(model.to_json())
#     	model.save_weights('best_model_weights.h5',overwrite=True)
#     	print("Best validation F1 score = {}".format(best_val_f1))
#     print()

Epoch 0
Training =>
0 [ 227 1095   31  222  582  887  584 1073 1204  361  443   75  396   43  864
  454 1187  815  749   54  863  907  849  280  817    3  501 1146  371 1213
  365  203   83  182  340   12  857  590  749  222  427  114  864 1130 1192
  883 1072  222  587  443  158  849  427  607  361 1121  280    0    6  676
  361  133 1127  610  905  361  712  887  440 1136  419  749  919 1096  883
  899  364  587  713  440  742  749  919 1096  393  427  869  365  637  676
 1127  701  180  396  123 1096]


1 [ 332  873 1031  749  757  222  396  382  596  482  120  183  493  905  159
  849 1126  412  729  676 1127 1087  333  873  713  722  222 1124 1128  886
  533  427  371 1121  365  132  706  512  905  823  396  657  849  144  222
  782  887  757  605 1127  543  938  905 1127  545  749  333  728  321  891
  905  849  393  605 1127  805  938 1192   61  863   73    0   74  456   83
  863   62    0  396   62  456  114  863   47    0  396   47  456  205  849
  905  607  699 1192 1106  393  605 1127  543  938  905 1127  545]


2 [ 338 1020 1127  223  905  923   76  695  588  868 1096 1126 1192  731  749
 1127 1046  624  507  321  820  905  222  587  443 1096  396 1127  702 1023
  905 1003  923 1143 1202  375 1127 1172  905  694  409 1146  743  693 1126
 1161 1131  587  335  921  844  340 1150  777  461  836 1127  223  905   13
  849  749  390  730  327  784  782  654 1146  418 1108  892  905  849  676
  992 1152  813  288  392  846  396 1146 1172 1131  846 1146  743  486 1012
  522 1146 1095  587  749  222  321 1095 1115 1189 1020  984 1176  768  676
  568 1131  645  321 1142  396 1023  378  782 1108  676  870 1052    0  325
  412  748  409  417 1132  834  375 1127  424  905  693 1204  222  396  700
 1199  538  789  684  597  887 1110 1126 1132  355  411  735  606  917  597
  887  745  743 1129  417  807  700  464]


3 [ 182 1127  691  513  905  222 1127  429  223  905 1081 1096  713  481  749
  340 1024  827  321  223  905  147  849  814  617  905 1086 1192 1185 1074
  676 1127  930 1065 1210  414   20  857  153  609   58  396   59  732 1127
  222  905 1135 1095  582  786  682  410  113  857 1146  410   20  857  203
   75  187 1135  938 1127 1039  749 1196  921  403 1192  731  396 1127  828
  905 1127  868  583 1192  481  279 1127 1046  938 1127  222  905  122  378
  757  602  682  116 1215    9  857  876    0   20  749   58 1146   44 1215
    8  857  876    0   55  749   59  321  757  222  749  122  378 1187 1024
 1204 1127  429  222  905  849  419  749 1127 1209   63  442   45 1215    8
  857  876    0   49  226 1127  223  905 1096  145  396  158  582  887  480
  605 1135 1143  145   25 1215    9  857  749   58  396   26 1215    7  857
  749   59  876   13  396   43  158   42 1215   11  857  749   58  396   41
 1215    6  857  749   59  876    0  115  396   55]


4 [ 155  964  598  887  529 1204  222]
5 [ 152  910  923   76 1099  868 1096  429  222  997  682  410   13 1146   62
  857  208  598  887 1157  355  222  396  366 1031  749  390  222  551  749
 1081 1096  262 1121 1096  640  340  596 1035 1204  222  442 1127  723  605
 1127  543  938  905 1127  545  216  676  340  890  372  905 1127  467  401
  221  690  172  598  887  379  222 1197  727  676  340  873  749 1127  801
 1006  690  244 1126  472  895  396  580 1031  749  757  222  131  159  849
  729  676  340 1158  873  333  713  722  223 1124 1128  944  533]


...

47 [ 329 1120  744  694  941 1126  470  895  396  580  461  379  222 1189  501
  849 1126 1192  695 1074  454 1192  612  729  917  720  676  340  801 1006
  873  548 1126 1031  749  895  396  580  441   83  864  905  361  910 1127
  160 1095  434 1173    0  222 1187  860  454 1073  613  749  894  581  729
  872   46 1215    7  857  501 1146  885  884  719   39 1215    8  857  280
  817    4  203  115]
48 [203 114]
49

 [175 634]
50 [243]
51 [ 133 1205 1127  995  905 1177  897  749 1212  849 1127  222  905  145  849
  860  552  414   12 1209  905  361  326  551 1187 1093 1072  501 1146 1212
  849  837  427 1127 1046 1143  142   61 1210  905  361 1127  222  905  145
  849  708  552  686 1138  390  610  905  397  749 1131 1185  907  849  465
  437 1045  922]
52 [ 222  482 1204  759  130    0  256  222 1215  298  782 1070  676 1127  763
 1095  427  588  365  222  552 1073  749 1096  159  396   31  454  887  158
  321  159  396   31  706  512  905  410  629  892  905  824  396  658  321
   75  864  907  158  849 1192  823 1197 1127  919  365 1192  506  905  824
  396  658  321  891  905  849  393  427  607  361  808  682 1214 1146  909
  673  159   54   54   54   54   35   31   54   47   49   43   64  158   47
   70   53  396   49]


53 [ 93 396  97]
54 [332]
55 [184]
56 [155 964]
57 [ 132  223 1192  419 1174  390  398 1010  905  119  848  788  396  115  848
 1208  556  417   60  228  645 1111 1126  390  377  741  599   14  848  788
  396  115  848 1208  905  397  708  883  610  910  222  605 1127  644  938
 1204  222  442  837  417 1083  417  956  358 1127  868 1187 1159 1157  855
    0 1189  742  757  222  605 1127  543  501 1146 1127  805  938  905 1127
  545  749 1065 1121 1096  203  106  227 1131  645  222  839 1192  819  443
   88  396   23  855  358  353  905  397]


58 [190 905 361 910 222]
59 [135 905 265]
60 [128]
61 [ 321  339  487  712  440  747  749 1095 1085  447  964  587  749 1000    0
  329 1120  744 1127  339  487  905 1095   29  383  222  749 1015 1146 1126
  905 1095  145 1189  501 1127  223  905  988  145  824  396  515  145  824
 1126  708 1127   30  339  487  433  676   20  692  269  587  749  222 1192
  573  443 1131  706  905  849  203  115  280    0    5]


KeyboardInterrupt: 